In [29]:
import json
import uuid

dataset_indicatori = json.load(open('Datasets/Indicatori/ds385_popolazione_indicatori_popolazione_famiglie_2011_breve-1.json'))
dataset_censimento = json.load(open('Datasets/Censimento_Milano_2011/sezioni_censimento_2011.geojson'))
fasce = json.load(open('Datasets/Indicatori/ds382_popolazione_popolazione_residente_caratteristiche_sezione_censimento_2011c_.json'))
famiglie = json.load(open('Datasets/Indicatori/ds383_popolazione_famiglie_residenti_caratteristiche_sezione_censimento_2011c.json'))

#Fasce preprocessing
fasce_ordered = {}
for i in fasce:
    obj = {}
    obj['0-5'] = int(i.get("Età < 5 anni"))
    obj['5-9'] = int(i.get("Età 5 - 9 anni"))
    obj['10-14'] = int(i.get("Età 10 - 14 anni"))
    obj['15-19'] = int(i.get("Età 15 - 19 anni"))
    obj['20-24'] = int(i.get("Età 20 - 24 anni"))
    obj['25-29'] = int(i.get("Età 25 - 29 anni"))
    obj['30-34'] = int(i.get("Età 30 - 34 anni"))
    obj['35-39'] = int(i.get("Età 35 - 39 anni"))
    obj['40-44'] = int(i.get("Età 40 - 44 anni"))
    obj['45-49'] = int(i.get("Età 45 - 49 anni"))
    obj['50-54'] = int(i.get("Età 50 - 54 anni"))
    obj['55-59'] = int(i.get("Età 55 - 59 anni"))
    obj['60-64'] = int(i.get("Età 60 - 64 anni"))
    obj['65-69'] = int(i.get("Età 65 - 69 anni"))
    obj['70-74'] = int(i.get("Età 70 - 74 anni"))
    obj['75-100'] = int(i.get("Età > 74 anni"))
    fasce_ordered[i.get("Numero sezione")] = obj

#Famiglie preprocessing
famiglie_ordered = {}
for i in famiglie:
    obj = {}
    obj['tot_famiglie'] = int(i.get("Famiglie residenti"))
    obj['1'] = int(i.get("Famiglie di 1 componente"))
    obj['2'] = int(i.get("Famiglie di 2 componenti"))
    obj['3'] = int(i.get("Famiglie di 3 componenti"))
    obj['4'] = int(i.get("Famiglie di 4 componenti"))
    obj['5'] = int(i.get("Famiglie di 5 componenti"))
    obj['6'] = int(i.get("Famiglie di 6 e oltre componenti"))
    famiglie_ordered[str(i.get("Numero sezione"))] = obj

#Ordering Indicatori + fasce
list = []
for i in dataset_indicatori:
    obj = {}
    for j in i:
        index = j.split(')',1)[0]
        if (index == "2" or index == "3" or index == "4" or index == "9" or index == "14" or index == "15" or index == "18" or index == "26" or  index == "27" or index == "28" or index == "35" or index == "40" or index == "41"):
            if (i.get(j) == None):
                obj[index] = ''
            elif (i.get(j) == ''):
                obj[index] = ''
            else:
                obj[index] = float(i.get(j).replace(',','.'))
        elif (index == "1"):
            obj["sez"] = int(i.get(j))
        else:
            continue
    obj["fasce"] = fasce_ordered.get(str(obj["sez"]))   
    obj["famiglie"] =famiglie_ordered.get(str(obj["sez"])) 
    list.append(obj)


#Data sorting and Indicatori + fasce generation code
indicatori = {"legenda":["1) Numero medio di residenti per edificio ad uso residenziale","2) Numero medio di abitazioni per edificio ad uso residenziale",
"3) Numero medio di abitazioni occupate da residenti per edificio ad uso residenziale","8) Percentuale di bambini in età prescolare","13) Percentuale di abitazioni non occupate",
"14) Percentuale di famiglie monogenitore","17) Percentuale di 65+ che vivono da soli","25) Giovani (0-14 anni) ogni 100 in età attiva (15-64anni)",
"26) Over65 enni ogni 100 persone in età attiva (15-64 anni) ","27) Percentuale di popolazione 15+ che percepisce reddito sul totale della popolazione 15+",
"34) Percentuale di popolazione residente che si sposta giornalmente per studio o lavoro","39) Numero medio di componenti delle famiglie",
"40) Percentuale di famiglie di un componente"],"indicatori":[]}
for i in list:
    obj_2 = {}
    obj_2["sez"] = i.get("sez")
    for j in range(2,45):
        if (i.get(str(j))== None):
            continue
        else:
            obj_2[j-1] = i.get(str(j))
    obj_2["fasce"] = i.get("fasce")   
    obj_2["famiglie"] = i.get("famiglie")          
    indicatori.get("indicatori").append(obj_2)

temp = []
for i in dataset_censimento.get("features"): temp.append(i.get("properties").get("SEZ"))
temp = max(temp)
ordered = []

#Ordering dataset_censimento
for i in range (0, temp+1):
    for j in dataset_censimento.get("features"):
        if (j.get("properties").get("SEZ") == i):
            ordered.append(j)

with open("Datasets/Censimento_Milano_2011/censimento_cleared.geojson", 'x') as outfile:
    json.dump(dataset_censimento, outfile, indent=1)
with open("Datasets/Indicatori/indicatori_cleared.json", 'x') as outfile:
    json.dump(indicatori, outfile, indent=1)

#Append indicatori to Censimento dataset
ordered_2 = []
for i in indicatori.get("indicatori"):
    for j in ordered:
        if (int(i.get("sez")) == int(j.get("properties").get("SEZ"))):
            j["indicatori"] = i
            ordered_2.append(j)
 
#Remove "Sez" data from Indicatori
for i in ordered_2: i.get("indicatori").pop("sez")
dataset_censimento["features"] = ordered_2

with open("Datasets/dataset_completo.geojson", 'x') as outfile:
    json.dump(dataset_censimento, outfile, indent=1)

In [4]:
from keplergl import KeplerGl

censimento = json.load(open('Datasets/dataset_completo.geojson'))
    
#Load Map config

map = KeplerGl(data={"censimento_milano_2011": censimento})
map

KeplerGl(data={'censimento_milano_2011': {'type': 'FeatureCollection', 'name': 'sezioni_censimento_2011', 'crs…

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [1]:
#Sezione-based randomic family generator
settori = {}
id_familiare = 1

for i in censimento.get("features"):

    #Remove malformed and/or empty sections
    if (i.get("properties").get("SEZ") == 54 or i.get("properties").get("SEZ") == 3233 or i.get("properties").get("SEZ") == 2785 or i.get("properties").get("SEZ") == 2842 or i.get("properties").get("SEZ") == 2873 or i.get("properties").get("SEZ") == 3042 or i.get("properties").get("SEZ") == 3581 or i.get("properties").get("SEZ") == 4145 or i.get("properties").get("SEZ") == 4339 or i.get("properties").get("SEZ") == 4797 or i.get("properties").get("SEZ") == 4982 or i.get("properties").get("SEZ") == 5669):
        continue
    if (i.get("properties").get("POP_2010") == 0 or i.get("indicatori").get("1") == '' or i.get("indicatori").get("1") == "0"):
        continue
    
    #Start printing sections data
    print("Codice sezione:",i.get("properties").get("SEZ"))
    POP_2010 = i.get("properties").get("POP_2010")
    edifici = round(POP_2010/float(i.get("indicatori").get("1")))
    tot_appartamenti = round(edifici*float(i.get("indicatori").get("3")))
    print("Popolazione:",POP_2010)
    print("Numero di edifici ad uso residenziale:", edifici)
    print("Numero di abitazioni totali ad uso residenziale:", tot_appartamenti)
    if (tot_appartamenti!=0):
        avg_ppa = POP_2010/tot_appartamenti
        print("Numero di abitanti per appartamento ad uso residenziale:", avg_ppa)
        print("Numero medio di componenti delle famiglie:", i.get("indicatori").get("39"))

    #ESTIMATED AGE GROUPS
    if (i.get("indicatori").get("25") != '' and i.get("indicatori").get("26") != ''):
        young = float(i.get("indicatori").get("25"))
        old = float(i.get("indicatori").get("26"))
        Y = old + young

        X = (100+Y)/100
        mid_percent = 100/X
        old_percent = old/X
        young_percent = young/X

        est_anziani = round((POP_2010/100)*(old_percent))
        est_giovani = round((POP_2010/100)*(young_percent))
        est_attivi = round((POP_2010/100)*mid_percent)

        #Clone POP_2010 and households for counting purposes
        popolazione = POP_2010
        appartamenti = tot_appartamenti

        print("[EST] Abitanti nella fascia 15-64:", est_attivi)
        #print("Abitanti fuori dalla fascia 15-64:",anziani+giovani)
        print("[EST] Abitanti < 15:", est_giovani)
        print("[EST] Abitanti > 65:", est_anziani)
        print("[EST] Over 65+ che vivono da soli:",round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27")))))
        print()

        #TODO: Generate age groups based on 'fasce' data
        settore = []
        #65+ old solo families generator
        for j in range (0,round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27"))))):
            fam = {}
            members = []
            obj = {}
            obj["uuid"] = str(uuid.uuid4())
            obj["age"] = "old"
            fam["family_id"] = id_familiare
            members.append(obj)
            fam["members"] = members
            id_familiare = id_familiare + 1
            anziani = anziani - 1
            settore.append(fam)

        #Remove created families from 65+
        popolazione = popolazione - round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27"))))
        appartamenti = appartamenti - round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27"))))

        #Families with children one parent generator
        if (i.get("indicatori").get("14") != "0"):
            max_fam = (giovani/100)*float(i.get("indicatori").get("14"))
            num = round(max_fam/avg_ppa)
            #NUM = Number of families with one parent
            for k in range (0,num):

                #Members removal code
                kids_number = round(max_fam/num)
                max_fam = max_fam - kids_number
                POP_2010 = POP_2010 - (kids_number + 1)
                giovani = giovani - kids_number 
                fam = {}
                members = []
                obj = {}
                obj["uuid"] = str(uuid.uuid4())
                obj["age"] = "active"
                members.append(obj)

                for kid in range(0,kids_number):
                    obj = {}
                    obj["uuid"] = str(uuid.uuid4())
                    obj["age"] = "kid"
                    members.append(obj)

                fam["family_id"] = id_familiare
                fam["members"] = members
                id_familiare = id_familiare + 1

                attivi = attivi - num
                popolazione = popolazione - len(fam.get("members"))
                settore.append(fam)

        if (giovani != 0):
            #Max families members / mean familiy size
            max_componenti = round(giovani*3)
            avg_fam = round(max_componenti/avg_ppa)

            if (avg_fam > giovani):
                num = giovani
            else:
                num = avg_fam

            #NUM = Number of families with two parents
            mean_kpf = round(giovani/num)
            for k in range(0,num):
                
                fam = {}
                members = []

                #Add parents
                for parent in range(0,2):
                    obj = {}
                    obj["uuid"] = str(uuid.uuid4())
                    obj["age"] = "mid"
                    members.append(obj)

                if (giovani < mean_kpf):
                    kids = giovani
                else: 
                    kids = mean_kpf

                for kid in range(0,kids):
                    obj = {}
                    obj["uuid"] = str(uuid.uuid4())
                    obj["age"] = "kid"
                    members.append(obj)
                
                POP_2010 = POP_2010 - (2+kids)
                giovani = giovani - kids
                attivi = attivi - 2

                fam["family_id"] = id_familiare
                fam["members"] = members
                id_familiare = id_familiare + 1
                appartamenti = appartamenti - 1 
                popolazione = popolazione - len(fam.get("members"))
                settore.append(fam)


        #Generate remaining families
        #65+ families
        for index in range (0, round(anziani/2)):
            fam = {}
            members = []

            if (anziani == 1):
                #1 old, 1 mid
                obj = {}
                obj["uuid"] = str(uuid.uuid4())
                obj["age"] = "old"
                members.append(obj)
                obj = {}
                obj["uuid"] = str(uuid.uuid4())
                obj["age"] = "mid"
                members.append(obj)
                anziani = anziani - 1
                attivi = attivi - 1
            else:
                #2 old
                for parent in range(0,2):
                    obj = {}
                    obj["uuid"] = str(uuid.uuid4())
                    obj["age"] = "old"
                    members.append(obj)
                anziani = anziani - 2

            fam["family_id"] = id_familiare
            fam["members"] = members
            id_familiare = id_familiare + 1
            appartamenti = appartamenti - 1 
            popolazione = popolazione - len(fam.get("members"))
            settore.append(fam)

        #Active families
        for index in range (0, round(attivi/2)):
            fam = {}
            members = []

            if (attivi == 1):
                obj = {}
                obj["uuid"] = str(uuid.uuid4())
                obj["age"] = "mid"
                members.append(obj)
                attivi = attivi - 1
            else:
                #2 old
                for parent in range(0,2):
                    obj = {}
                    obj["uuid"] = str(uuid.uuid4())
                    obj["age"] = "mid"
                    members.append(obj)
                attivi = attivi - 2

            fam["family_id"] = id_familiare
            fam["members"] = members
            id_familiare = id_familiare + 1
            appartamenti = appartamenti - 1
            popolazione = popolazione - len(fam.get("members")) 
            settore.append(fam)
        
        settori[i.get("properties").get("SEZ")] = {"ace":i.get("properties").get("ACE"),"families":settore}




NameError: name 'censimento' is not defined

In [26]:
with open("household_layer.json", 'x') as outfile:
    json.dump(settori, outfile, indent=1)